https://docs.google.com/presentation/d/1nITW2Mh6jqwl8ouO36GlRrCkVCcVtYY3syI6fVgKyi4/edit?usp=sharing

Omplir group.json

# Sistemes de Recomanació

En aquest primer lliurament es programarà un **sistema de recomanació**, que posarà en correspondència un *usuari* amb *artistes* en funció de les seves preferències i interessos. 
En aquesta ocasió, implementareu un sistema de recomanació que indiqui quin artista escoltar.

## Abans de començar


**\+ Durant la pràctica, solament es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

*Nota: A més de les que ja es troben presents en la 1a cel·la i funcions natives de Python*

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica que serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades. Durant els testos, els paràmetres (i específicament `df`) no contindran les mateixes dades que en aquest notebook, si bé si seran del mateix tipus! Per tant, no us refieu de què tinguin, per exemple, el mateix nombre de files.

## Testos automàtics

Com ja sabeu, les pràctiques es fan a través de Github Classroom. Podeu treballar-hi lliurement i es recomana que feu commits sovint, per tal que els canvis quedin reflectits de forma estructurada i modular.

Normalment treballareu a la branca `master`, però podeu fer fins a 3 cops al dia un `commit` (o `merge` de `master`) a la branca `test`. Això provocarà que es llencin un seguit de proves sobre el vostre codi, en podreu veure el resultat a la següent web: http://grade-me.education

Penseu que aquests testos són un subconjunt, petit, dels que realment farem servir per avaluar. Per tant, us recomanem que aprofiteu al màxim els 3 intents diaris, que us serviran per comprovar que els formats d'entrada i sortida siguin correctes, a més d'alguns testos bàsics de correcte funcionament.

# Les dades

En aquest i futur notebooks farem servir dades reals corresponents a *Last.fm dataset*. 

# Data loading

### **En aquestes cel·les no feu cap modificació**

Carrega les dades en un DataFrame Pandas.

In [1]:
import pandas as pd
import numpy as np
import zipfile
import pickle
from os.path import join, dirname

def locate(*path):
    base = globals().get('__file__', '.')
    return join(dirname(base), *path)

def unzip(file):
    zip_ref = zipfile.ZipFile(locate(file), 'r')
    zip_ref.extractall(locate('data'))
    zip_ref.close()

if __name__ == '__main__':
    unzip('dades_alumnes.zip')

    original_df_artists = pd.read_csv('data/artists_names.csv', index_col='musicbrainzID')
    original_df_users = pd.read_csv('data/user_profile_train.csv', index_col='userID')
    original_df_user_artists = pd.read_csv('data/user_artist_plays_train.csv', index_col=0)
    
    print(original_df_artists.shape, original_df_users.shape, original_df_user_artists.shape)
    print(original_df_artists, original_df_users, original_df_user_artists)

/Users/marcosplazagonzalez/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(160112, 1) (251543, 4) (10215268, 3)
                            artistName
musicbrainzID                         
0                      betty blowtorch
1                            die Ärzte
2                    melissa etheridge
3                            elvenking
4                 juliette & the licks
5                red hot chili peppers
6                               magica
7              the black dahlia murder
8                          the murmurs
9                           lunachicks
10                    walls of jericho
11                      letzte instanz
12                           goldfrapp
13                          horrorpops
14                        the butchies
15                       jack off jill
16                    babes in toyland
17                    dropkick murpfys
18                       all my faults
19                            le tigre
20                          schandmaul
21                               edguy
22                 maximum

In [2]:
if __name__ == '__main__':
    df_user_artists = original_df_user_artists[original_df_user_artists.userID < 1000]
    print(df_user_artists.shape)
    print(len(df_user_artists['userID'].unique()), len(df_user_artists['musicbrainzID'].unique()))

(28987, 3)
707 6544


## Anàlisis de les dades

El primer que haurem de fer és analitzar les dades mitjançant diferents funcions.

In [225]:
def count_users(df):
    """
    Retorna el nombre d'usuaris en el dataframe
    
    :param df: DataFrame dels usuaris
    :return : Enter, nombre d'usuaris
    """
    return df.shape[0] #Donat el df dels usuaris retornem el numero de files

def count_artits(df):
    """
    Retorna el nombre d'artistes en el dataframe
    
    :param df: DataFrame dels artistes
    :return : Enter, nombre d'artistes
    """
    return df.shape[0] #Donat el df dels artistes retornem el numero de files

def get_users(df):
    """
    Retorna els ids dels usuaris
    
    :params df: DataFrame df_user_artists
    :return : Llista, tupla, pd.Series o indexos de pandas amb les ids
    """
    return pd.Series(list(df.index)) #Donat el df dels usuaris els ids coincideixen
                                     #amb la columna dels index del df

def get_artits(df):
    """
    Retorna els ids dels artites
    
    :params df: DataFrame df_user_artists
    :return : Llista, tupla, pd.Series o indexos de pandas amb les ids
    """
    return pd.Series(list(df.index)) #Donat el df dels artistes els ids coincideixen
                                     #amb la columna dels index del df
def total_reproductions(df):
    """
    Retorna la quantitat de reproduccions totals guardades al dataframe
    
    :params df: DataFrame df_user_artists
    :return : Enter, nombre de reproduccions
    """
    return df.loc[:,'plays'].sum() #Donat el df_user_artists agafem la columna 'plays' i la sumem sencera

In [226]:
if __name__ == '__main__':
    print('count_users: {}'.format(count_users(original_df_users)))
    print('count_artits: {}'.format(count_artits(original_df_artists)))
    print('get_users: {}'.format(get_users(original_df_users)))
    print('get_artits: {}'.format(get_artits(original_df_artists)))
    print('total_reproductions: {}'.format(total_reproductions(original_df_user_artists)))

count_users: 251543
count_artits: 160112
get_users: 0         310519
1         109228
2          35296
3         291224
4         151053
5         267414
6          63446
7          80577
8         187410
9         339146
10          5238
11         38215
12        297228
13         93162
14        344303
15        204135
16        151747
17        110934
18         99701
19        129229
20        149519
21        155584
22         96965
23         43787
24         43909
25         56327
26        251540
27         39262
28        158021
29        158937
           ...  
251513     93670
251514    235634
251515    232980
251516    314257
251517    139939
251518     89881
251519    114338
251520    259653
251521     63623
251522    220838
251523     51399
251524    263682
251525    261846
251526    217313
251527    147508
251528       117
251529    109601
251530    307185
251531    336881
251532    211221
251533     43472
251534    130769
251535     66696
251536    315919
251537     98

# Implementació

Recordeu, seguiu els pydoc i compliu amb el que diuen!

El primer que haurem de fer és construir una matriu que ens serveixi, d'alguna forma, com a indicatiu de preferències de cada persona. Per tal efecte, construirem una matriu $m\times n$, de $m$ usuaris per $n$ artistes (items), on cada entrada $i,j$ serà el nombre de vegades que la persona $i$ a escoltat l'artista $j$.

<img src="./img/Mat.png">


In [227]:
from scipy.sparse import csr_matrix, dok_matrix
from pandas.api.types import CategoricalDtype

def to_dense(array):
    """
    Accepta una csr_matrix, dok_matrix o matrix i la converteix en una 
    np.array normal, densa.
    
    :param array: Array a convertir
    :return: np.array densa, sense cap dimensió de tamany 1
    """
    try:
        array = array.todense()
    except:
        pass
    
    return np.array(array).squeeze()
    
def build_counts_table(df):
    """
    Retorna una csr_matrix on les columnes són els `items`, les files `user_id` i els valors
    el nombre de vegades que un usuari ha escoltat un `artist`
    
    :param df: DataFrame original després de creuar-lo
    :return: Una tupla constistent de:
        * La csr_matrix descrita
        * Els indexos corresponents a cada fila (el userID de la fila `i` corresponent a l'element `i` d'aquesta array)
        * Les columnes corresponents a cada columna (el musicbrainzID de la columna `j` correspon a l'element `j` d'aquesta array)
    """
    # Ids, sense repeticions i ordenats
    user_ids = CategoricalDtype(sorted(df.userID.unique()), ordered=True)
    music_ids = CategoricalDtype(sorted(df.musicbrainzID.unique()), ordered=True)
    
    # Conversió a csr
    row = df.userID.astype(user_ids).cat.codes
    col = df.musicbrainzID.astype(music_ids).cat.codes
    sparse_matrix = csr_matrix((df["plays"], (row, col)), \
                           shape=(user_ids.categories.size, music_ids.categories.size))

    return sparse_matrix, user_ids, music_ids

In [228]:
def get_count(counts, indexes, columns, user_id, artist_id):
    """
    Exemple: donat un ID d'usuari i un ID d'artista, retorna
        el valor corresponent de la matriu `counts`
    
    :param counts, indexes, columns: Tupla retornada per `build_counts_table`
    :param user_id: ID de l'usuari
    :param artist_id: ID de l'artista
    :return: Enter amb el nombre de vegades que s'ha escoltat
    """
    row = indexes.categories.get_loc(user_id)
    col = columns.categories.get_loc(artist_id)
    return counts[row, col]

In [7]:
def top_active_users(counts, indexes, columns, n):
    """
    Exemple: Retorna els ids dels n usuaris que més reproduccions han acumulat
    
    :param counts, indexes, columns: Tupla retornada per `build_counts_table`
    :param n: Quanitat d'usuaris
    :return: Llista, tupla o pd.Series de userID dels n usuaris
    """
    # Operate with the sparse matrix, convert to dense the result (as it has much fewer entries)
    sums = to_dense(counts.sum(axis=1))
    # Get indices
    indices = sums.argsort()
    return indexes.categories[indices[-n:]]

def top_reproduced_artits(counts, indexes, columns, n):
    """
    Exemple: Retorna els ids dels n artistes més escoltats
    
    :param counts, indexes, columns: Tupla retornada per `build_counts_table`
    :param n: Quanitat d'artistes
    :return: Llista, tupla o pd.Series de artistID dels n artistes
    """
    # Operate with the sparse matrix, convert to dense the result (as it has much fewer entries)
    sums = to_dense(counts.sum(axis=0))
    # Get indices
    indices = sums.argsort()
    return columns.categories[indices[-n:]]

In [8]:
if __name__ == '__main__':
    counts, indexes, columns = build_counts_table(df_user_artists)
    print(counts.shape)
    
    count = get_count(counts, indexes, columns, 111, 3323)
    print(count)
    
    top_users = top_active_users(counts, indexes, columns, 5)
    print(top_users)
    
    top_artists = top_reproduced_artits(counts, indexes, columns, 5)
    print(top_artists)

(707, 6544)
2796
Int64Index([220, 603, 1, 877, 557], dtype='int64')
Int64Index([246, 217, 158, 1757, 578], dtype='int64')


### Mesurament de similituds

El primer pas per poder recomanar és definir una funció de similitud entre vectors. Siguin $x, y$ vectors, les següents són funcions típiques de similitud:

* Distància euclidea (inversa): https://en.wikipedia.org/wiki/Euclidean_distance

$$sim(x, y) = \frac{1}{1 + \sqrt{\sum_i(x_i-y_i)^2}}\in [0, 1]$$

* Similitud cosinus: https://en.wikipedia.org/wiki/Cosine_similarity

$$sim(x, y) = \frac{x\cdot y}{||x||\hspace{0.1cm} ||y||} \in [-1,1]$$

* Correlació de Pearson: https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

$${\displaystyle sim(x,y)={\frac {\sum _{i=1}^{n}(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})}{{\sqrt {\sum _{i=1}^{n}(x_{i}-{\bar {x}})^{2}}}{\sqrt {\sum _{i=1}^{n}(y_{i}-{\bar {y}})^{2}}}}}} \in [-1,1] \\ \text{On }\bar{x} = \frac{1}{n} \sum^n_i x_i\text{ la mitja (i anàlogament per y)}$$

Per aquesta part, però, **us demanem que implementeu** la següent funció de similitud:

* Jaccard: https://en.wikipedia.org/wiki/Jaccard_index

$$sim(x, y) = {{|x \cap y|}\over{|x \cup y|}} $$

Aquesta funció mesura el nombre d'elements en comú dels dos vectors (numerador) i el nombre total d'elements diferents que hi ha entre els dos vectors.
Per exemple: 

$$
x = [1, \text{NaN}, 2, 3, \text{NaN}, 5] \\
y = [2, 1, \text{NaN}, \text{NaN}, \text{NaN}, \text{NaN}] \\
sim(x, y) = {{|x \cap y|}\over{|x \cup y|}} = {1 \over{5}} = 0.2
$$

Per implementar-la **no podeu fer servir bucles** ni cap funció que calculi directament el resultat:

* *scipy.spatial.distance.jaccard*
* *sklearn.metrics.jaccard_similarity_score*
* I d'altres semblants

<hr>

Tant per aquesta última com per les altres funcions, tingueu en compte que tenen particularitats. Algunes consideren valors negatius per els exemples oposats, d'altres poden donar problemes quan el denominador és 0, etc.

In [9]:
import numpy as np

def similarity(x, y):
    """
    Similitud entre x i y segons Jaccard
    
    :param x: Primer vector, com a np.array
    :param y: Segon vector, com a np.array
    :return : Escalar (float) corresponent a la similitud
    """
    cardinalitat_interseccio = np.sum(np.logical_and(x!=0,y!=0)) #Calculem la interseccio dels vectors x,y...
    cardinalitat_unio = np.sum(np.logical_or(x!=0,y!=0)) #i a continuació la unió
    return cardinalitat_interseccio / cardinalitat_unio 

In [10]:
if __name__ == '__main__':
    x = np.array([1, 1, 1, 0., 0])
    y = np.array([0, 3, 0, 1, 5])
    print(similarity(x, y))

0.2


### Matriu de similituds

Per fer recomanació col·laborativa existeixen dues opcions, fer un recomanador basat en usuaris o un en ítems:

* Recomanador basat en usuaris:
Considera la matriu $M\times N: \text{usuaris}\times\text{items}$, per recomanar t'hauràs de basar en les similituds entre els usuaris.

* Recomanador basat en items:
Considera la matriu $M\times N: \text{items}\times\text{usuaris}$, per recomanar t'hauràs de basar en les similituds entre els ítems.

Construeix una matriu de mida $M\times M$ on cada posició $i,j$ indica la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. En canvi, si l'estàs fent basat en ítems, `matriu[2, 3]` contindrà la similitud entre l'ítem 2 i el 3.

In [241]:
from tqdm import tqdm_notebook

def similarity_matrix(similarity_function, counts):
    """
    Retorna una matriu dok_matrix (sparse) de mida M x M on la posició
    (i, j) indica la similitud entre els usuaris `i` i `j` (resp. items).
    
    No necessàriament totes les posicions han de ser obligatòriament omplertes.
    Una bona implementació considerarà una matriu triangular, tenint-ho en compte
    en les següents funcions.
    
    :param similarity_function: Fnció que calcularà la similitud 
        entre usuaris (resp. ítems)u
    :param counts: csr_matrix que conté el nombre de vegades que 
        un usuari ha escoltat a un `artistID`
    :return : Matriu numpy de mida M x M amb les similituds.
    """    
    # Matrix cosine
    if similarity_function is None:
        # Base similarity matrix (all dot products)
        # Replace this with A.dot(A.T).toarray() for sparse representation
        base_matrix = counts.toarray()
        base_matrix_prod = np.dot(base_matrix, base_matrix.T)

        # Squared magnitude of preference vectors (number of occurrences)
        square_mag = np.diag(base_matrix_prod)

        # Inverse squared magnitude
        inv_square_mag = 1 / square_mag

        # if it doesn't occur, set it's inverse magnitude to zero (instead of inf)
        inv_square_mag[np.isinf(inv_square_mag)] = 0

        # inverse of the magnitude
        inv_mag = np.sqrt(inv_square_mag)

        # Cosine similarity (elementwise multiply by inverse magnitudes)
        cosine = base_matrix_prod * inv_mag
        cosine = cosine.T * inv_mag
        
        # Fill diagonal values with 0
        np.fill_diagonal(cosine, 0)
        
        return cosine
    
    else:
        n = counts.shape[0] # Resp. [1] per items
        matrix = dok_matrix((n, n)) # Empty matrix, needs to be filled

        counts_array = counts.toarray() #Així podem treballar normalment amb la matriu counts

        for i in range(n):
            for j in range(i,n): #Matrix -> triangular
                if i == j: 
                    matrix[i,j] = 0 #0s en la diagonal principal
                else:
                    similar = similarity_function(counts_array[i],counts_array[j]) #Omplim la matriu de similaritats
                    matrix[i,j] = similar    

        return matrix #Retornem la dok_matrix

Per cridar aquesta funció, el primer paràmetre pot ser:

* Si `similarity_function` no és `None`: `similarity_function` és una funció que rep dos np.array i calcula la similitud (tipus similarity(x, y)). Utilitzant ~5000 usuaris i amb una bona implementació triga ~45min.
* Opcionalment (no és obligatori fer-ho, penseu en Kaggle) podeu programar una funció que treballi específicament amb matrius (i no vectors). Si ho feu, cal gestionar-ho quan es rep `None`. No totes les funcions anteriorment anomenades són fàcils (ni intuïtives, ni hi caben en memòria) d'aplicar en forma matricial. Triga uns 5s.

In [242]:
import pickle

if __name__ == '__main__':
    try:
        with open('similarities.pkl', 'rb') as fp:
            similarities = pickle.load(fp)
            print(similarities.toarray())
    except:
        similarities = similarity_matrix(
            similarity_function=similarity,
            counts=counts
        )
        
        with open('similarities.pkl', 'wb') as fp:
            pickle.dump(similarities, fp, pickle.HIGHEST_PROTOCOL)

[[0.         0.         0.01111111 ... 0.04395604 0.01190476 0.        ]
 [0.         0.         0.03296703 ... 0.02083333 0.         0.        ]
 [0.         0.         0.         ... 0.08695652 0.01123596 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.03296703 0.01052632]
 [0.         0.         0.         ... 0.         0.         0.01176471]
 [0.         0.         0.         ... 0.         0.         0.        ]]


### Generació de prediccions

Per fer recomanació col·laborativa, necessitem una funció que ens doni un valor de quant bona seria la recomanació. En el nostre cas i amb les nostres dades, volem una funció que ens indiqui quants cops escoltaria un usuari un artista donat.

* Si esteu fent un recomanador basat en usuaris, la puntuació per a un usuari $u$ i l'artista $i$ és

$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{p\neq u,r_{p,i}>0} sim(u, p)\cdot r_{p,i}}{\sum_{p\neq u,r_{p,i}>0} sim(u, p)}$$

On $r_{u,i}$ indica el nombre de vegades que l'usuari $u$ ha escoltat l'l'ítem $i$.

És a dir, per cada usuari $p$ diferent de $u$ si aquest usuari ha escoltat algun cop el producte $i$, la similitud entre $p$ i $u$ multiplicada pel nombre de vegades que l'usuari $p$ ha escoltat l'l'ítem $i$ ($r_{p,i}$).

Pondera't per la suma de les similituds.

* Anàlogament, si està basat en ítem, la puntuació per a un usuari $u$ i ítem $i$ és

$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{j\neq i,r_{u,j}>0} sim(i, j)\cdot r_{u,j}}{\sum_{j\neq i,r_{u,j}>0} sim(i, j)}$$

On $r_{u,i}$ indica el nombre de vegades que l'usuari $u$ ha escoltat l'ítem $j$.

És a dir, per cada ítem $j$ diferent de $i$ si l'usuari al qui recomanem ha escoltat l'artista $j$, la similitud entre $i$ i $j$ multiplicada pel nombre de vegades que l'usuari al qui recomanem $u$ ha escoltat l'ítem $j$ ($r_{u,j}$)

Pondera't per la suma de les similituds.

Fixeu-vos que, sigui quin sigui el cas, al final estem fent el producte vectorial entre dos vectors. Concretament, el producte vectorial entre les similituds i les compres. Fes una funció que calculi aquest resultat:

In [296]:
def score(counts, indexes, columns, similarities, user, item):
    """
    Extreu les similituds i el nombre de vegades que un usuari ha escoltat un artista
    (resp. nombre de vegades que cada usuari ha escoltat l'artista) i:
    
    * Si estàs implementant basat en usuaris:
        Donades les similituds i el nombre de vegades que l'usuari ha escoltat
        cada artista, retorna la predicció que indica quants cops escoltara 
        l'usuari un nou artista.
        
    * Si estàs implementant basat en items:
        Donades les similituds i el nombre de vegades que l'artista a recomanar ha
        estat escoltat per cada usuari, retorna la predicció que indica quants cops
        escoltaria l'usuari un nou artista.        
    
    :param counts, indexes, columns: Tupla retornada per `build_counts_table`
    :param similarities: Matriu de similituds
    :param user: ID de l'usuari per la predicció
    :param item: ID de l'artista / item per la predicció    
    :return : Retorna un escalar (float) amb la predicció
    
    """
    #I consider that user valid value is inside of the interval -> [1,707]
    if user <= 0 and user >= counts.shape[0]:
        return "Not a valid user"
    
    #Fem una de les implementacions: o basat en usuaris o basat en items??
    counts = counts.copy() #Matrix 707x6544 which contains number of plays
    counts = counts.toarray()
    repr_values = counts[:,item]
    aux = {i:repr_values[i] for i in range(len(repr_values))}
    #mask_repr = test_counts[:,item]>0 #For every row lets check the item. We want the values > 0.
    
    #predict_users = np.where(mask_repr) #Okay 

    num = 0
    den = 0

    for p in range(len(aux)):
        if aux[p] != 0 and not user == p:#We want to assure not to compute sim(u,u)
                print(user,p)
                num+=similarities[user,p]*aux[p]#test_counts[p,item] 
                den+=similarities[user,p]
        
    if den == 0:#Avoids division by zero 
        return np.nan
    
    return num/den

In [297]:
if __name__ == '__main__':
    print(score(counts, indexes, columns, similarities, 111, 123))

111 259
111 637
10.0


###### Feu una funció que donat un usuari calculi per cada item que no ha escoltat la puntuació d'aquest. La funció retorna els $N$ items més ben puntuats.

In [257]:
def recommend_n_items(counts, indexes, columns, similarities, user, N):
    """
    Donat un usuari calcula per cada ítem/artista que no ha escoltat la puntuació d'aquest. 
    La funció retorna els $N$ ítems més ben puntuats.
    
    :param counts, indexes, columns: Tupla retornada per `build_counts_table`
    :param similarities: Matriu de similituds
    :param user: Identificador de l'usuari
    :param N: Nombre d'ítems que volem que siguin recomanats.
    :return : Llista amb els IDs dels ítems recomanats
    """
    if not 0 <= user <= counts.shape[0]:
        return "Not a valid user"
    
    usuari = counts[user].toarray()[0]
    dict_usuari = {i:usuari[i] for i in range(len(usuari))}
    
    score_aux = {}
    
    if 0<= N <= len(usuari):
        for i in dict_usuari.keys():
            if dict_usuari[i] != 0:
                score_aux[i] = score(counts,indexes,columns,similarities,user,i)
    else:
        return "N is bigger than the number of items!"
        
    #retornar el top_n_recommends
    temp = sorted(score_aux.items(),key=lambda x: x[1],reverse=True)
    score_clean = [temp[x][0] for x in range(len(temp)) if not np.isnan(temp[x][1])]
    
    return score_clean[:N]

In [258]:
if __name__ == '__main__':
    print(recommend_n_items(counts, indexes, columns, similarities, 111, 10))

[986, 795, 998, 712, 388, 233, 226, 783, 2536, 2550]


### Possibles millores 

Per implementar millores, dupliqueu el notebook i feu-ho en la còpia!

Deixeu aquest notebook amb la implementació base.

**1) És Jaccard la millor mesura de similitud per les nostres dades?**
Jaccard no té en compte la cardinalitat, únicament els elements no nuls. És, per les dades de reproduccions, una aproximació adequada?
Si ho és, creus que el càlcul de recomanació col·laborativa és el més adhient? Mira el punt 3

**2) Utilització completa de les dades:**
Fer servir `df_original` tindrà (possiblement) resultats més fiables, però trigarà molt més que amb la versió reduida `df`. Pots canviar tant el nombre de dades utilitzades com quines dades es seleccionen.

**3) Normalització: Prediccions escalades al domini de l'usuari:**
Els usuaris escolten en diferent mesura als artites, uns més vegades, d'altres menys. Fent servir la següent formula, escalem la predicció a la mitja de l'usuari:
$$pred(u, i) = \hat{r}_{u,i} = \bar{r_u} + \frac{\sum_{p\neq u,r_{p,i}>0} sim(u, p)\cdot (r_{p,i}-\bar{r_i})}{\sum_{p\neq u,r_{p,i}>0} sim(u, p)}$$
on $\bar{r_u}$ és la mitjana de compres de l'usuari *u* i $\bar{r_i}$ la mitja de reproduccions de l'artista *i*.
    
**4) Valor del nombre d'elements codificats: **
Redueix la similitud entre els usuaris si el nombre de reproduccions és baix o descarta (en entrenament) aquells usuaris amb un petit nombre d'artistes escoltats.

**5) Augment de la similitud: **
Incrementeu el pes als usuaris que són realment similars (~ = 1)

**6) Selecció d'usuaris semblants: **
Només s'utilitza un subconjunt d'usuaris similars, descartant tots aquells usuaris poc similars.


Totes aquestes tècniques es poden aplicar d'igual manera en la recomanació col·laborativa per usuaris o ítems.

# Kaggle

Teniu la competició a la següent URL:

https://www.kaggle.com/t/4a3fd7b1e44c40a5a208c6da21a23d05

Si heu seguit tots els pydoc, el següent codi funcionarà directament sense canvis (@30min).

In [209]:
if __name__ == '__main__':
    try:
        from tqdm import tqdm_notebook
    except:
        def tqdm_notebook(a, *_, **__): return a
    
    # Fetch kaggle public data and merge
    kaggle_df_user_artists = pd.read_csv('data/user_artist_plays_public_kaggle.csv')
    merged_df_user_artists = pd.concat((df_user_artists, kaggle_df_user_artists))
    
    # Obtain counts
    merged_counts, merged_indexes, merged_columns = build_counts_table(merged_df_user_artists)
    
    # Similarity
    try:
        with open('merged_similarities.pkl', 'rb') as fp:
            merged_similarities = pickle.load(fp)
    except:
        merged_similarities = similarity_matrix(
            similarity_function=similarity,
            counts=merged_counts
        )
        
        with open('merged_similarities.pkl', 'wb') as fp:
            pickle.dump(merged_similarities, fp, pickle.HIGHEST_PROTOCOL)
    
    results = pd.DataFrame(columns=['user_id', 'music_id'])
    
    for idx, user in enumerate(tqdm_notebook(kaggle_df_user_artists.userID.unique())):
        music_ids = recommend_n_items(merged_counts, merged_indexes, merged_columns, merged_similarities, user, 10)
        results.loc[idx] = (user, ' '.join(str(x) for x in music_ids))
        
    results.to_csv('submission.csv', index=None)

/Users/marcosplazagonzalez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':
